In [1]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [5]:
data = pd.read_csv('OV2.csv', on_bad_lines='skip', sep=';')
data = pd.DataFrame(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7386 entries, 0 to 7385
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              7386 non-null   object 
 1   Streams           7386 non-null   int64  
 2   Viewers           7386 non-null   int64  
 3   Dominant_streams  7386 non-null   int64  
 4   Dominant_ratio    7386 non-null   float64
 5   Tags              7386 non-null   object 
 6   Name              7386 non-null   object 
 7   Title             7386 non-null   object 
 8   Beginning         7386 non-null   object 
 9   Unnamed: 9        52 non-null     object 
 10  Hour              7386 non-null   int64  
 11  Minute            7386 non-null   int64  
 12  Weekday           7386 non-null   int64  
 13  Moving_streams    7377 non-null   object 
dtypes: float64(1), int64(6), object(7)
memory usage: 808.0+ KB


In [6]:
X = data[['Hour','Minute','Weekday']]
#y = data[['Streams','Dominant_streams','Viewers', 'Dominant_ratio']]

y = data[['Moving_streams']]
#y['Moving_viewers'] = y['Moving_viewers'].str.replace(',', '.').astype(float)
y['Moving_streams'] = y['Moving_streams'].str.replace(',', '.').astype(float)
y = y.drop(y.index[0:10])
X = X.drop(X.index[0:10])

C:\Users\tymot\AppData\Local\Temp\ipykernel_16860\2667274269.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['Moving_streams'] = y['Moving_streams'].str.replace(',', '.').astype(float)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, shuffle=False)
y_test = y_test.reset_index()

In [14]:

xgb_model = xgb.XGBRegressor(n_estimators = 1000)

#xgb_model.fit(X, y, eval_set=[(X, y), (X, y)],verbose=100)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

y_pred = y_pred.astype(int)
y_test['Moving_streams'] = y_test['Moving_streams'].astype(int)
#print(y_pred)
#print(y_test['Moving_streams'])


wynik = 0
wynik_list =[]
#For bigger games
# for i in range(len(y_test)):
#     temp = max(y_pred[i],y_test['Moving_streams'][i]) / min(y_pred[i],y_test['Moving_streams'][i])
#     temp = ((temp * 100) - 100) * 10
#     temp = 100 - temp
#     if temp > 0:
#         wynik += temp/100
#         wynik_list.append(temp)
#     else:
#         wynik += 0
#         wynik_list.append(0)

for i in range(len(y_test)):
    if y_test['Moving_streams'][i] == y_pred[i]:
        wynik += 1
        wynik_list.append(100)
    elif y_pred[i] == y_test['Moving_streams'][i] + 1 or  y_pred[i] == y_test['Moving_streams'][i] - 1:
        wynik += 0.5
        wynik_list.append(50)
    else:
        wynik += 0 
        wynik_list.append(0)

# For bigger games
# df = pd.DataFrame(y_pred)
# df['should'] = y_test['Moving_streams']
# df['result'] = wynik_list
# df.columns = ['is','should','result']
# display(df)

print(wynik/len(y_test))

df = pd.DataFrame(y_pred)
df['should'] = y_test['Moving_streams']
df['result'] = wynik_list
df.columns = ['is','should','result']
display(df)



0.2635135135135135


,is,should,result
0,8,9,50
1,9,9,100
2,9,8,50
3,9,9,100
4,9,9,100
...,...,...,...
69,7,13,0
70,8,13,0
71,8,13,0
72,8,13,0


In [10]:

new_data = pd.DataFrame({'Hour': [17], 'Minute': [0], 'Weekday': [4]})
prediction = xgb_model.predict(new_data)

print('Score: ', prediction)

Score:  [0.47920606]


In [15]:
xgb_model.feature_importances_

array([0.80591214, 0.02951659, 0.1645713 ], dtype=float32)

In [19]:
from sklearn.ensemble import IsolationForest

X = data[['Hour','Minute','Weekday','Streams']]

isof = IsolationForest(n_estimators=500)
isof.fit(X)

new_data = pd.DataFrame({'Hour': [9], 'Minute': [12], 'Weekday': [2], 'Streams': [5]})
prediction = isof.predict(new_data)

print('Wynik: ', prediction)


Wynik:  [1]
